<a href="https://colab.research.google.com/github/micah-shull/AI_Agents/blob/main/066_Staged_Execution_with_Review.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 🛠️ Staged Execution: Plan, Review, Execute

Sometimes we need more control than simply rolling back on failure. **Staged execution** gives us that control by letting us:

- ✅ **Plan** a series of actions in advance
- 🔍 **Review** them for safety, correctness, and intent
- 🚀 **Execute** only after validation

This pattern is especially valuable when:

- Some actions are **more reversible** than others
- You want to involve a **human-in-the-loop** or a more capable AI for final approval
- There's a **high cost or risk** to incorrect execution

With staged execution, agents can behave more like thoughtful collaborators—**asking before they act** rather than begging for forgiveness later.




#### ✅ 1. **Why Staged Execution Exists**

Staged execution is all about **preparing actions in advance**, **reviewing them for safety**, and then executing *only if approved*. It gives you a chance to:

* Prevent errors *before* they happen (not just recover after).
* Involve human or high-trust AI review before committing.
* Handle **partially irreversible** actions with more care.

This is especially helpful for high-risk operations, like:

* Sending emails on behalf of execs.
* Booking high-cost resources.
* Modifying customer accounts.

---

#### 🛠️ 2. **The `StagedActionEnvironment` Class**

This extends your normal environment to include **reviewable transactions**:

```python
env.stage_actions(task_id)
```

Creates a "transaction" — a set of actions you want to propose.

```python
env.review_transaction(task_id)
```

Allows a high-trust LLM (or human) to inspect the full plan before executing it. It answers:

* Are all actions necessary?
* Are they in a safe order?
* Could they cause harm?

✅ The goal: **explicit approval before commitment.**

---

#### 🤖 3. **The Role of the Review LLM**

The environment uses an *LLM as a reviewer* to pre-filter actions before possibly sending them to a human. This makes it scalable — humans only step in when really needed.

* This reviewer LLM isn't executing the actions.
* It’s acting like a **safety officer** checking the checklist.

---

#### 📋 4. **The Example: Scheduling a Meeting**

The sample code shows how to:

1. Check availability immediately.
2. **Stage** both calendar creation and email send.
3. Pass both through review **before executing**.
4. Only commit the transaction if it’s safe.

This mirrors **real-world workflows**:

> Draft → Review → Approve → Execute

---

### ✨ Why This Pattern Matters

* **Precision + Accountability**: You know *exactly* what’s about to happen.
* **Safety Net**: Mistakes are caught before damage occurs.
* **Human-AI Collaboration**: The agent isn’t reckless — it seeks permission when needed.



In [ ]:
class StagedActionEnvironment(Environment):
    def __init__(self):
        self.staged_transactions = {}
        self.llm = None  # High-capability LLM for review

    def stage_actions(self, task_id: str) -> ActionTransaction:
        """Create a new transaction for staging actions."""
        transaction = ActionTransaction()
        self.staged_transactions[task_id] = transaction
        return transaction

    def review_transaction(self, task_id: str) -> bool:
        """Have LLM review staged actions for safety."""
        transaction = self.staged_transactions.get(task_id)
        if not transaction:
            raise ValueError(f"No transaction found for task {task_id}")

        # Create a description of staged actions
        staged_actions = [
            f"Action: {action.__class__.__name__}\nArgs: {args}"
            for action, args in transaction.actions
        ]

        # The safest way to do this would be to send it for human review,
        # but we can also imagine having a more capable AI system review it before the human
        # to minimize the number of reviews that the human has to do. The more capable AI can review
        # and reject potentially problematic actions earlier.

        review_prompt = f"""Review these staged actions for safety:

        Task ID: {task_id}

        Staged Actions:
        {staged_actions}

        Consider:
        1. Are all actions necessary for the task?
        2. Could any action have unintended consequences?
        3. Are the actions in a safe order?
        4. Is there a safer way to achieve the same goal?

        Should these actions be approved?
        """

        response = self.llm.generate(review_prompt)

        # If approved, notify the human and ask if
        # they want to proceed
        return "approved" in response.lower()




### ✅ **What’s Inside the Full Tool**

#### `__init__`

* Initializes the environment.
* Stores:

  * A dictionary of staged transactions keyed by `task_id`.
  * A reference to a more powerful LLM for safety review.

#### `stage_actions(task_id)`

* Initializes a new `ActionTransaction` object.
* Registers it under the given `task_id` for future lookup.
* Returns the transaction so actions can be added to it.

#### `review_transaction(task_id)`

* Retrieves the staged transaction.
* Converts the list of staged actions into a **human-readable description**.
* Constructs a **review prompt** that asks the LLM to check:

  * Necessity
  * Safety
  * Order
  * Alternatives
* Interprets the LLM’s response to determine whether to proceed.

---

### 🧠 Why This Is One Cohesive Tool

This tool does **two tightly related things**:

1. **Staging**: Safely prepares a sequence of actions for execution — no side effects yet.
2. **Reviewing**: Requests an external evaluation (by LLM or human) to ensure actions are safe before executing.

These are *conceptually linked* operations and belong in the same class — especially in a high-trust environment like scheduling, finance, or HR systems.

---

### 📌 What This Tool Does *Not* Do

This class **does not execute** or **commit** the actions — that's left to the consumer of this tool, like:

```python
if env.review_transaction(task_id):
    await transaction.execute()
    transaction.commit()
```

So, it's meant to be used **in coordination** with tools like `ActionTransaction` and domain-specific tools (e.g., `send_email`, `create_event`), as you’ve already seen.




In this case, the `__init__` (a class constructor) is used instead of decorators like `@register_tool` because this class isn’t a **tool** in the usual sense — it’s an **environment utility** or **infrastructure component** that **manages behavior** rather than **executes specific tasks**.

Let’s unpack why:

---

### 🧩 `@register_tool` is for discrete, callable tools

When you see something like:

```python
@register_tool(description="Send an email")
def send_email(...): ...
```

That function:

* Is **meant to be invoked by an agent**
* Performs a **single, focused action**
* Is typically stateless
* Has a well-defined input/output interface

It’s a **tool** the agent can select from its action registry during reasoning.

---

### 🛠️ `StagedActionEnvironment` is not a tool — it’s a system service

This class is doing something different:

* It manages **transactions** across multiple steps
* It stores **state** (`self.staged_transactions`)
* It is designed to **coordinate** tools, not to be one

It’s part of the **agent's execution runtime**, not something the agent chooses from a list. Think of it like a database connection or logging system — it’s part of the foundation, not something you register as a user-facing capability.

---

### 🔍 Why use `__init__` here?

The `__init__` method is used:

* To initialize state (`self.staged_transactions`)
* To inject dependencies (`self.llm`)
* To set up reusable behaviors (`stage_actions`, `review_transaction`)

It reflects that the class will have **internal state over time**, unlike a tool, which is typically **stateless**.

---

### 🧠 Analogy: Tool vs. Infrastructure

| Purpose           | Example                       | Decorator?         | Stateful? |
| ----------------- | ----------------------------- | ------------------ | --------- |
| Single-use action | `send_email()`                | ✅ `@register_tool` | ❌ No      |
| System behavior   | `StagedActionEnvironment`     | ❌ No               | ✅ Yes     |
| Agent service     | `ActionTransaction`, `Logger` | ❌ No               | ✅ Yes     |

---

### ✅ Bottom line:

`@register_tool` is for **exposed, agent-callable actions**.
`__init__` is for **system scaffolding** — reusable services and orchestrators that support agent logic.





### 🔁 **The Goal of This Pattern**

You don’t want actions to be executed immediately. Instead, you:

1. **Stage** them (like drafting a plan).
2. **Review** them (to check for safety, correctness, or approval).
3. **Execute** only if the review passes.

---

### 🧱 Breakdown of the Code

#### `class StagedActionEnvironment(Environment):`

This defines a new type of **environment** that supports staging + reviewing actions before running them.

---

#### `def __init__(self):`

* `self.staged_transactions`: A dictionary to keep track of multiple staged action sets, each associated with a unique `task_id`.
* `self.llm`: A high-capability LLM that will later be used to **review** staged plans.

---

#### ✅ `stage_actions(self, task_id: str) -> ActionTransaction`

* This function starts a new "plan" for a given task.
* It creates a new `ActionTransaction` (a container for reversible actions).
* It **doesn't run anything yet** — just queues actions for review.

**🧠 Analogy:** Think of this like drafting your to-do list, not doing the tasks yet.

---

#### 👀 `review_transaction(self, task_id: str) -> bool`

* This method **inspects the queued actions** (before they run).
* It retrieves the actions for the given task.
* Then it **describes** them — it creates readable summaries:

  ```python
  f"Action: {action.__class__.__name__}\nArgs: {args}"
  ```
* The goal is to prepare a list of what the agent *intends* to do, which the LLM (or a human) can review.

**What’s Missing Here?**

* The actual LLM call to review the action list. That part would typically look like this:

  ```python
  decision = self.llm(
      Prompt([
          {"role": "system", "content": "You are a safety reviewer..."},
          {"role": "user", "content": f"Should we execute the following actions?\n{formatted_list}"}
      ])
  )
  return decision == "approve"
  ```

---

### 💡 Why This Matters

This pattern gives you:

* **Control**: Nothing runs unless approved.
* **Auditability**: You know what was planned and what got changed.
* **Intervention**: Humans or high-trust AI can catch mistakes before they cause harm.




This **staged execution pattern** is a **special-purpose tool**, just like the others you’ve built. It fits seamlessly into the modular, orchestrator-driven agent architecture you’ve been developing.

Here’s how it integrates:

---

### 🛠️ **Staged Action as Just Another Tool**

You treat it like any other modular tool:

| Tool                     | Purpose                                              |
| ------------------------ | ---------------------------------------------------- |
| `check_calendar`         | Look for conflicts before booking                    |
| `schedule_meeting`       | Draft a new meeting time                             |
| `draft_email`            | Compose an email, don't send it yet                  |
| **`stage_actions`**      | Queue up planned operations for review (not yet run) |
| **`review_transaction`** | Ask LLM or human to approve the staged actions       |
| `send_email`             | Finalize and send the email after approval           |

---

### 🔁 **How It Works Together — Orchestrator View**

Here’s a simplified version of the flow:

```python
task_id = "follow_up_with_timothy"

# Stage multiple actions
transaction = env.stage_actions(task_id)
transaction.add(schedule_meeting_tool, time="Friday 2PM", invitees=["Timothy"])
transaction.add(draft_email_tool, recipient="Timothy", subject="New Meeting Time", body="See updated time...")

# Ask for approval before executing
if env.review_transaction(task_id):
    await transaction.execute()
    transaction.commit()
else:
    print("Plan rejected — taking no action.")
```

---

### 🧠 **Why This Pattern Matters**

* **Safety**: Nothing executes without a second look.
* **Predictability**: The orchestrator knows the steps and can bundle them logically.
* **Modularity**: Each tool focuses on a narrow concern, making the system composable and easy to extend.

---

This approach is especially useful when the stakes are higher (like sending sensitive emails, making financial moves, or scheduling exec meetings). It mimics how real teams operate: draft → review → approve → act.




This final part ties the whole *Staged Execution with Review* pattern together and really brings home the **systems thinking** behind robust, trustworthy AI agents. Here's what to focus on:

---

## 🔍 What to Pay Attention To

### 1. **Pre-Execution Review Using a Prompt**

You're using an LLM (or eventually a human) to **simulate an approval workflow** — this is powerful!

```python
review_prompt = f"""Review these staged actions for safety:
...
Should these actions be approved?
"""
```

* 📌 **Why this matters**: You're *not* blindly executing logic. You're checking:

  * Necessity
  * Potential side effects
  * Safe sequencing
  * Better alternatives

This mimics real-world workflows: "Plan first → Review → Execute if approved."

---

### 2. **Tiered Review: LLM then Human**

```python
# The safest way... human review
# ...more capable AI system review before the human...
```

This is **layered safety**:

* ✳️ Fast, automated filtering via LLM
* ✅ Final gatekeeping via human

This helps **minimize false positives** and **optimize human attention** — the AI acts as a smart assistant *filtering out noise*.

---

### 3. **Structured Execution After Approval**

The final function `schedule_team_meeting()` is a **realistic example** of how you use staged execution in a task pipeline:

```python
transaction.add(create_event, ...)
transaction.add(send_email, ...)
```

Key Points:

* 🔧 Immediate actions (e.g., checking availability) happen *outside* the transaction.
* 🧱 Potentially risky or irreversible actions (event creation, email sending) are **staged**.
* 🧑‍⚖️ Only after review, they’re **executed and committed**.

---

### 4. **You’re Building a Safety-Critical Agent**

This pattern is important when the **consequences of a mistake matter**:

* Sending an email to the wrong person
* Overwriting a calendar event
* Making financial transactions

This approach helps create **robust, auditable, and trustworthy AI systems**.

---

## 💡 Summary: What You’ve Learned to Spot

| Focus Area                          | What to Look For                                       |
| ----------------------------------- | ------------------------------------------------------ |
| 🔁 Transaction Logic                | What gets staged vs executed immediately               |
| 🧠 Review Prompt Strategy           | How safety & correctness are evaluated                 |
| 👥 Approval Workflow                | Tiered AI → Human approach to review                   |
| 💥 Reversibility vs Irreversibility | What happens if a step fails, and how it’s mitigated   |
| ⚙️ Modular Tools + Orchestration    | How agents assemble, queue, and review tasks modularly |




In [ ]:
async def schedule_team_meeting(env: StagedActionEnvironment,
                              attendees: List[str],
                              duration: int):
    """Schedule a team meeting with safety checks."""
    task_id = str(uuid.uuid4())
    transaction = env.stage_actions(task_id)

    # Check availability (execute immediately)
    available_slots = calendar.check_availability(attendees, duration)
    if not available_slots:
        return {"error": "No available time slots"}

    best_slot = available_slots[0]

    # Stage the event creation
    transaction.add(create_event,
                   title="Team Meeting",
                   time=best_slot,
                   duration=duration)

    # Draft email (execute immediately)
    email_draft = email.draft_message(
        to=attendees,
        subject="Team Meeting",
        body=f"Team meeting scheduled for {best_slot}"
    )

    # Stage the email send
    transaction.add(send_email,
                   draft_id=email_draft.id)

    # Review staged actions...send to human review
    # or more capable AI for initial filtering
    if env.review_transaction(task_id):
        await transaction.execute()
        transaction.commit()
        return {"status": "scheduled"}
    else:
        return {"status": "rejected"}


## 🧰 Tool vs. 🏗️ Infrastructure (or Environment Utility)

### 🧰 **Tool**

Think of a *tool* like a button the agent can press to do something **specific**.

* **Purpose:** A single, well-defined task
* **How it's used:** The agent *calls* it directly when it needs to perform an action
* **Examples:**

  * `send_email(to, subject, body)`
  * `extract_invoice_data(text)`
  * `categorize_expense(description)`
* **Registered via:** `@register_tool(...)`

**Analogy:** Like a wrench in a toolbox — you grab it when you want to tighten a bolt. One task. Simple.

---

### 🏗️ **Infrastructure / Environment Utility**

Now contrast that with *infrastructure* or an *environment utility*. These aren't actions the agent directly requests — they’re **systems that manage** how tools are used or how things behave behind the scenes.

* **Purpose:** Orchestrate, track, or support tools
* **How it’s used:** The agent or developer relies on it to manage **execution, safety, coordination, review, or rollback**
* **Examples:**

  * `StagedActionEnvironment`: stages actions and asks for approval before running them
  * `ActionTransaction`: tracks a series of reversible actions
  * `ToolUsageLogger`: tracks every tool call for audit purposes

**Analogy:** Like a factory assembly line manager — they don’t weld the parts themselves, but they *coordinate who does what and in what order* safely and efficiently.

---

## 📦 Environment Utility = Behind-the-Scenes Agent Infrastructure

An **Environment Utility** is a specialized helper system that:

* Exists inside the agent’s **runtime environment**
* Supports **safe execution**, **auditing**, **planning**, or **multi-step coordination**
* Might control:

  * When a tool can run
  * Whether to allow or reject a transaction
  * How to roll back errors
  * Whether to send something for review

It's called a *utility* because it's not the *main tool*, but it adds critical **functionality**, like logging, staging, or reviewing actions.

---

### 👨‍🔧 Example:

Let’s say an agent needs to:

1. Schedule a meeting
2. Draft an email
3. Send the email

#### If it just runs the tools directly:

* Great — simple and fast
* But if something goes wrong mid-way, you’ve got a half-scheduled meeting and no email

#### If it uses `StagedActionEnvironment`:

* It first **stages** the meeting and email actions
* Then a human or smarter LLM **reviews them**
* If approved, the system runs them safely, in order
* If something fails, it **undoes** earlier steps

That’s infrastructure. The agent’s using **tools** — but the **environment utility** makes sure they’re used **safely**.

---

## ✅ Summary Table

| Type                    | Does...                                      | Who uses it?      | Example                                        |
| ----------------------- | -------------------------------------------- | ----------------- | ---------------------------------------------- |
| **Tool**                | Performs a focused task                      | Called by agent   | `send_email`, `categorize_invoice`             |
| **Environment Utility** | Supports, coordinates, or reviews tool usage | Built into system | `StagedActionEnvironment`, `ActionTransaction` |





Let’s integrate your `ActionTransaction` class into a simplified version of a full **orchestrator setup**, including the `StagedActionEnvironment`. The environment will **manage transactions**, and the orchestrator agent can **use that environment to stage, review, and execute actions** safely.

Here’s how they all fit together in **one clean flow**:

---

### ✅ Step-by-Step Code Sketch

```python
import uuid
from datetime import datetime

# Assume ReversibleAction class is already defined
# Assume tool functions like `create_event`, `send_email` are also wrapped as ReversibleAction instances

class ActionTransaction:
    def __init__(self):
        self.actions = []
        self.executed = []
        self.committed = False
        self.transaction_id = str(uuid.uuid4())

    def add(self, action, **args):
        if self.committed:
            raise ValueError("Transaction already committed")
        self.actions.append((action, args))

    async def execute(self):
        try:
            for action, args in self.actions:
                result = action.run(**args)
                self.executed.append(action)
        except Exception as e:
            await self.rollback()
            raise e

    async def rollback(self):
        for action in reversed(self.executed):
            await action.undo()
        self.executed = []

    def commit(self):
        self.committed = True


# 🧠 This is the environment that manages safety
class StagedActionEnvironment:
    def __init__(self, review_llm):
        self.staged_transactions = {}
        self.llm = review_llm  # LLM that reviews staged actions

    def stage_actions(self, task_id: str) -> ActionTransaction:
        transaction = ActionTransaction()
        self.staged_transactions[task_id] = transaction
        return transaction

    def review_transaction(self, task_id: str) -> bool:
        transaction = self.staged_transactions.get(task_id)
        if not transaction:
            raise ValueError("No transaction found")

        staged_actions = [
            f"Action: {a.__class__.__name__}\nArgs: {args}"
            for a, args in transaction.actions
        ]

        review_prompt = f"""
        Review these staged actions for safety:

        Task ID: {task_id}

        Staged Actions:
        {staged_actions}

        Consider:
        1. Are all actions necessary?
        2. Any unintended consequences?
        3. Safe execution order?
        4. Safer alternatives?

        Should these actions be approved?
        """

        response = self.llm.generate(review_prompt)
        return "approved" in response.lower()
```

---

### 🤖 Orchestrator Example Function

This simulates how an **orchestrator agent** might use the environment:

```python
async def handle_meeting_scheduling(env: StagedActionEnvironment, attendees, duration):
    task_id = str(uuid.uuid4())
    transaction = env.stage_actions(task_id)

    # Lookup availability (this might be a normal tool, not staged)
    best_slot = calendar.find_best_time(attendees, duration)

    # Stage creating the calendar event
    transaction.add(create_event, title="Weekly Sync", time=best_slot, duration=duration)

    # Stage sending an email
    transaction.add(send_email, to=attendees, subject="Meeting Invite", body=f"Join us at {best_slot}")

    # Review before final execution
    if env.review_transaction(task_id):
        await transaction.execute()
        transaction.commit()
        return {"status": "success", "time": best_slot}
    else:
        return {"status": "rejected", "reason": "Review failed"}
```

---

### 🔧 Where This Fits

| Component                    | Purpose                                |
| ---------------------------- | -------------------------------------- |
| `ReversibleAction`           | A single reversible task               |
| `ActionTransaction`          | Groups reversible actions for rollback |
| `StagedActionEnvironment`    | Coordinates the review and execution   |
| `Orchestrator (agent logic)` | Uses the above to safely execute plans |

---

### 🧠 Why It Matters

* **Modular:** Each layer is responsible for *one thing* (just like a good software design)
* **Safe:** Human/AI review happens *before* execution
* **Reversible:** If something breaks, we can undo it
* **Extendable:** You can plug in your own LLM reviewer or even a human interface


